## Problem 1
Perform an appropriate train-test split on the data. In your report, justify why <br>
you chose to do the split in the way you did. In further problems, only ever <br>
perform exploratory data analysis on the training data, as testing data should <br>
never be used to draw hypotheses.

In [95]:
import pandas as pd
from sklearn.model_selection import train_test_split

RANDOM_STATE = 1

raw_df = pd.read_csv("life_expectancy.csv")
# Test size of 0.25 because this was recommended in a lecture.
train_df: pd.DataFrame
test_df: pd.DataFrame
train_df, test_df = train_test_split(raw_df, test_size=0.25, random_state=RANDOM_STATE)

## Problem 2.a
Identify the variable that has the strongest linear relationship with the <br>
target variable. Do so by calculating and reporting the Pearson product-moment <br>
correlation coefficient between this variable and the target variable (LEB) in <br>
your report. Hint: use functions for doing this from the Pandas dataframe <br>
package. <br> <br>

Our correlation study showed that the strongest positvie linear correlation with <br>
LEB was Human Development Index (value) with a **Pearson coefficient of 0.916981**

In [96]:
# Calculate the Pearson correlation coefficient between each feature and the target LEB under column "Life Expectancy at Birth, both sexes (years)"
corr_df = train_df.corr("pearson", numeric_only=True)

# Human developement index, HDI, had the strongest positive linear correlation with LEB at 0.916981
# This means that increase in HDI means increase in LEB
print(corr_df["Life Expectancy at Birth, both sexes (years)"].sort_values(ascending=False))

Life Expectancy at Birth, both sexes (years)                                       1.000000
Human Development Index (value)                                                    0.916981
Expected Years of Schooling, female (years)                                        0.810141
Expected Years of Schooling (years)                                                0.796385
Median Age, as of 1 July (years)                                                   0.792761
Expected Years of Schooling, male (years)                                          0.775236
Mean Years of Schooling, female (years)                                            0.744041
Mean Years of Schooling (years)                                                    0.738286
Mean Years of Schooling, male (years)                                              0.723811
Population with at least some secondary education, female (% ages 25 and older)    0.688005
Population with at least some secondary education, male (% ages 25 and older)   

## Problem  2.b
Construct a single-variable linear model using this variable. In your re-
port, report the coefficient of determination, and the coefficients of your model,
including the intercept. Also, produce a scatter plot showing the LEB as the
function of the variable you identified. Plot the regression line over the scatter
plot.


In [97]:
from sklearn.linear_model import LinearRegression
import numpy as np
from numpy import ndarray

x_hdi_values = np.array(train_df["Human Development Index (value)"]).reshape(-1,1)
y_leb_values = np.array(train_df["Life Expectancy at Birth, both sexes (years)"])

# Create a linear regression model
model = LinearRegression()
model.fit(x_hdi_values, y_leb_values)

r_score = model.score(x_hdi_values, y_leb_values)
model_intercept = model.intercept_
model_slope = model.coef_

print(f"R-squared value: {r_score}")
print(f"Intercept: {model_intercept}")
print(f"Slope: {model_slope}")


R-squared value: 0.8408542110825198
Intercept 34.6583231013562
slope [51.33734509]
